## Evaluation of clustering (shape-based)

- Use labeled data collected from anomaly_patterns_clustering_shape.ipynb.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import jsonlines
import glob
from collections import defaultdict

In [3]:
for f in glob.glob("../samples/clustering_anomaly_patterns/*.jsonl"):
    with jsonlines.open(f) as reader:
        aggr = defaultdict(int)
        aggr2 = defaultdict(int)
        for obj in reader:
            aggr[obj["anomaly_pattern"]] += 1
            aggr2[obj["anomaly_position"]] += 1
    if len(aggr) < 1:
        continue
    display(f, aggr, aggr2)

'../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221030-162851.jsonl'

defaultdict(int,
            {'Level shift down': 982,
             'Single spike': 458,
             'Level shift up': 550,
             'Multiple spikes': 321,
             'Single dip': 376,
             'Fluctuations': 86,
             'Other normal': 364,
             'Steady increase': 711,
             'Transient level shift up': 51,
             'Transient level shift down': 17,
             'White noise': 112,
             'Steady decrease': 52,
             'Sudden increase': 67,
             'Multiple dips': 12,
             'Sudden decrease': 49})

defaultdict(int,
            {'anomaly_during_fault': 3045,
             'anomaly_outside_fault': 695,
             'no_anomaly': 468})

'../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221028-172414.jsonl'

defaultdict(int,
            {'Level shift up': 34,
             'Single dip': 49,
             'Single spike': 65,
             'Other normal': 12,
             'Multiple spikes': 34,
             'Steady increase': 8,
             'Transient level shift down': 4,
             'Transient level shift up': 10,
             'Level shift down': 10,
             'White noise': 8,
             'Multiple dips': 2,
             'Sudden increase': 2,
             'Fluctuations': 3,
             'Steady decrease': 3})

defaultdict(int,
            {'anomaly_during_fault': 149,
             'anomaly_outside_fault': 75,
             'no_anomaly': 20})

In [4]:
import numpy as np
import pandas as pd
import random
import scipy.interpolate
import scipy.stats

In [5]:
import sys
sys.path.append('../')

from tsdr import tsdr

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
fpath = "../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221030-162851.jsonl"

samples: dict = {}
time_series_by_case: dict[tuple[str, str], list[tuple[str, np.ndarray]]] = defaultdict(list)
with jsonlines.open(fpath) as reader:
    for obj in reader:
        time_series_by_case[(obj["chaos_type"], obj["chaos_comp"])].append((obj["metric"], np.array(obj["time_series"])))
        
        key = (obj["chaos_type"], obj["chaos_comp"], obj["metric"])
        samples[key] = {"series": np.array(obj["time_series"], dtype=np.float64)}
        apos, apattern = obj["anomaly_position"], obj["anomaly_pattern"]
        if apos == "no_anomaly" or apattern in ["White noise", "Other normal"]:
            samples[key].update({
                "anomaly_type": "type0",
                "anomaly_pattern": "normal",
                "anomaly_position": apos,
            })
        else:
            match apattern:
                # Type 1
                case "Level shift down" | "Level shift up" | "Steady decrease" | "Steady increase" | "Sudden decrease" | "Sudden increase":
                    samples[key].update({
                        "anomaly_type": "type1",
                        "anomaly_pattern": apattern,
                        "anomaly_position": apos,
                    })
                # Type 2
                case "Fluctuations" | "Multiple dips" | "Multiple spikes" | "Single dip" | "Single spike" | "Transient level shift down" | "Transient level shift up":
                    samples[key].update({
                        "anomaly_type": "type2",
                        "anomaly_pattern": apattern,
                        "anomaly_position": apos,
                    })

In [7]:
from meltria.priorknowledge import priorknowledge
from joblib import Parallel, delayed


pk = priorknowledge.new_knowledge(
    target_app="train-ticket",
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewalres": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

def _hdbscan_clustering(pk, time_series, dist_type="sbd"):
    metric_name_to_values = {metric: scipy.stats.zscore(values) for metric, values in time_series}
    _, clustering_info = tsdr.Tsdr("residual_integral", **{
        "step2_clustering_method_name": "dbscan",
        "step2_dbscan_min_pts": 2,
        "step2_dbscan_dist_type": dist_type,  # 'pearsonr' or 'sbd'
        "step2_dbscan_algorithm": "hdbscan",  # 'dbscan' or 'hdbscan'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": 'medoid',  # 'medoid' or 'maxsum'
    }).reduce_multivariate_series(pd.DataFrame(metric_name_to_values), pk, n_workers=1)
    return clustering_info

In [8]:
clustering_infos = Parallel(n_jobs=-1)(delayed(_hdbscan_clustering)(pk, ts) for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [9]:
pearsonr_clustering_infos = Parallel(n_jobs=-1)(delayed(_hdbscan_clustering)(pk, ts, "pearsonr") for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [10]:
pd.options.display.max_rows = None
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

In [11]:
from itertools import combinations

def _create_df_from_clustering_infos(_clustering_infos: list) -> pd.DataFrame:
    eval_stat: list[tuple[str, str, int, str, int, int, int, int, int]] = []
    for i, ((chaos_type, chaos_comp), time_series) in enumerate(time_series_by_case.items()):
        clustering_info = _clustering_infos[i]
        for i, (representative_metric, sub_metrics) in enumerate(clustering_info.items(), start=1):
            atype_positives, atype_negatives = 0, 0
            apos_positives, apos_negatives = 0, 0
            metrics = set([representative_metric] + sub_metrics)
            for u, v in combinations(metrics, 2):
                u_atype: str = samples[chaos_type, chaos_comp, u]["anomaly_type"]
                u_apos: str = samples[chaos_type, chaos_comp, u]["anomaly_position"]
                v_atype: str = samples[chaos_type, chaos_comp, v]["anomaly_type"]
                v_apos: str = samples[chaos_type, chaos_comp, v]["anomaly_position"]
                if u_atype == v_atype:
                    atype_positives += 1
                else:
                    atype_negatives += 1
                if u_apos == v_apos:
                    apos_positives += 1
                else:
                    apos_negatives += 1

            eval_stat.append((chaos_type, chaos_comp, i, representative_metric, atype_positives, atype_negatives, apos_positives, apos_negatives, len(metrics)))

    return pd.DataFrame(eval_stat, columns=["chaos_type", "chaos_comp", "cluster_no", "rep", "type_positives", "type_negatives", "pos_positives", "pos_negatives", "total_metrics"]).reset_index().set_index(["chaos_type", "chaos_comp", "cluster_no"])

eval_df = _create_df_from_clustering_infos(clustering_infos)
eval_df

index                        rep                          type_positives  type_negatives  pos_positives  pos_negatives  total_metrics
chaos_type       chaos_comp          cluster_no                                                                                                                                       
pod-memory-hog   ts-preserve-service 1              0              s-ts-preserve_request_duration_seconds          1               2              3              0              3     
                                     2              1          c-ts-preserve-service_fs_reads_bytes_total          3               0              3              0              3     
                                     3              2   m-ts-preserve-service_java_lang_GarbageCollect...         10               0             10              0              5     
                                     4              3   m-ts-preserve-service_java_lang_Threading_Curr...        141             135            193             83             24     
                                     5              4   m-ts-preserve-service_Tomcat_RequestProcessor_...         28               0             28              0              8     
                                     6              5   m-ts-preserve-service_java_lang_Compilation_To...         15               0             15              0              6     
                                     7              6            c-ts-preserve-service_memory_mapped_file          6               0              6              0              4     
                                     8              7      c-ts-preserve-service_memory_working_set_bytes          6               0              6              0              4     
                                     9              8                     c-ts-preserve-service_last_seen          3               0              3              0              3     
                                     10             9   c-ts-preserve-service_network_receive_packets_...          6               0              6              0              4     
                                     11            10   m-ts-preserve-service_Tomcat_Servlet_requestCount          6               0              6              0              4     
                                     12            11   m-ts-preserve-service_Tomcat_GlobalRequestProc...          3               0              3              0              3     
                                     13            12   m-ts-preserve-service_java_lang_GarbageCollect...         10               0             10              0              5     
                                     14            13   m-ts-preserve-service_Tomcat_GlobalRequestProc...          3               0              3              0              3     
                                     15            14    m-ts-preserve-service_java_lang_MemoryPool_Valid         55               0             55              0             11     
                                     16            15   m-ts-preserve-service_java_lang_ClassLoading_L...          6               0              6              0              4     
                                     17            16   m-ts-preserve-service_Tomcat_WebResourceRoot_l...          3               0              3              0              3     
                                     18            17   m-ts-preserve-service_java_lang_GarbageCollect...         16              12             28              0              8     
                                     19            18   m-ts-preserve-service_java_lang_Threading_Thre...         45               0             45              0             10     
                                     20            19   m-ts-preserve-service_java_lang_MemoryPool_Usa...          3               0              3              0              3     
                       

In [12]:
eval_df_sum = eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["type_positives"].sum() / (x["type_positives"].sum() + x["type_negatives"].sum()))
eval_df_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.50
                  ts-order-mongo         0.50
                  ts-station-service     0.63
                  ts-train-service       0.48
                  ts-travel-service      0.71
pod-memory-hog    ts-consign-mongo       0.55
                  ts-order-service       0.79
                  ts-preserve-service    0.71
                  ts-station-service     0.50
                  ts-train-mongo         0.57
pod-network-loss  ts-auth-mongo          0.77
                  ts-basic-service       0.64
                  ts-price-mongo         0.88
                  ts-travel-mongo        0.83
                  ts-travel2-service     0.56
dtype: float64

In [13]:
eval_df_pos_sum = eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["pos_positives"].sum() / (x["pos_positives"].sum() + x["pos_negatives"].sum()))
eval_df_pos_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.52
                  ts-order-mongo         0.73
                  ts-station-service     0.62
                  ts-train-service       0.57
                  ts-travel-service      0.69
pod-memory-hog    ts-consign-mongo       0.70
                  ts-order-service       0.71
                  ts-preserve-service    0.84
                  ts-station-service     0.61
                  ts-train-mongo         0.54
pod-network-loss  ts-auth-mongo          0.72
                  ts-basic-service       0.63
                  ts-price-mongo         0.83
                  ts-travel-mongo        0.79
                  ts-travel2-service     0.63
dtype: float64

In [14]:
eval_df_sum.groupby(["chaos_type"]).mean()

chaos_type
pod-cpu-hog         0.57
pod-memory-hog      0.63
pod-network-loss    0.74
dtype: float64

In [15]:
eval_df_pos_sum.groupby(["chaos_type"]).mean()

chaos_type
pod-cpu-hog         0.63
pod-memory-hog      0.68
pod-network-loss    0.72
dtype: float64

In [16]:
eval_df_sum.mean()

0.6422814480435749

### Comparison with the existing method (FluxRank)

In [17]:
def _fluxrank_clustering(pk, time_series, dist_type="pearsonr"):
    metric_name_to_values = {metric: values for metric, values in time_series}
    _, clustering_info = tsdr.Tsdr("residual_integral", **{
        "step2_clustering_method_name": "dbscan",
        "step2_dbscan_min_pts": 1,
        "step2_dbscan_dist_type": dist_type,  # 'pearsonr' or 'sbd'
        "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": 'medoid',  # 'medoid' or 'maxsum'
    }).reduce_multivariate_series(pd.DataFrame(metric_name_to_values), pk, n_workers=1)
    return clustering_info


In [18]:
fluxrank_clustering_infos = Parallel(n_jobs=-1)(delayed(_fluxrank_clustering)(pk, ts) for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [19]:
fluxrank_sbd_clustering_infos = Parallel(n_jobs=-1)(delayed(_fluxrank_clustering)(pk, ts, "sbd") for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [20]:
fluxrank_eval_df = _create_df_from_clustering_infos(fluxrank_clustering_infos)
pd.options.display.max_rows = None
fluxrank_eval_df

index                        rep                          type_positives  type_negatives  pos_positives  pos_negatives  total_metrics
chaos_type       chaos_comp          cluster_no                                                                                                                                       
pod-memory-hog   ts-preserve-service 1              0                        s-ts-preserve_requests_count           0              0               0              0             1     
                                     2              1                  s-ts-preserve_request_errors_count           0              0               0              0             1     
                                     3              2              s-ts-preserve_request_duration_seconds           0              0               0              0             1     
                                     4              3   m-ts-preserve-service_java_lang_Memory_NonHeap...        2157           1938            4095              0            91     
                                     5              4   m-ts-preserve-service_Tomcat_WebModule_request...          55              0              55              0            11     
                                     6              5   m-ts-preserve-service_Tomcat_GlobalRequestProc...           3              0               3              0             3     
                                     7              6   m-ts-preserve-service_jmx_scrape_duration_seconds           0              0               0              0             1     
                                     8              7   m-ts-preserve-service_java_lang_OperatingSyste...           0              0               0              0             1     
                                     9              8   m-ts-preserve-service_java_lang_Memory_HeapMem...           0              0               0              0             1     
                                     10             9   m-ts-preserve-service_java_lang_GarbageCollect...           0              0               0              0             1     
                                     11            10   m-ts-preserve-service_Tomcat_GlobalRequestProc...           3              0               3              0             3     
                                     12            11   m-ts-preserve-service_Tomcat_GlobalRequestProc...           0              0               0              0             1     
                                     13            12   m-ts-preserve-service_Tomcat_Manager_processin...           0              0               0              0             1     
                                     14            13   m-ts-preserve-service_java_lang_Threading_Curr...           1              0               1              0             2     
                                     15            14   m-ts-preserve-service_java_lang_OperatingSyste...           0              0               0              0             1     
                                     16            15         c-ts-preserve-service_cpu_cfs_periods_total           0              0               0              0             1     
                 ts-order-service    1             16   m-ts-order-service_java_lang_GarbageCollector_...        2036           1792            3828              0            88     
                                     2             17      m-ts-order-service_jmx_scrape_duration_seconds           0              0               0              0             1     
                                     3             18    m-ts-order-service_Tomcat_Manager_processingTime           0              0               0              0             1     
                                     4             19   m-ts-order-service_Tomcat_RequestProcessor_req...           0              0               0              0             1     
                       

In [21]:
fluxrank_eval_df_sum = fluxrank_eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["type_positives"].sum() / (x["type_positives"].sum() + x["type_negatives"].sum()))
fluxrank_eval_df_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.96
                  ts-order-mongo         0.82
                  ts-station-service     0.98
                  ts-train-service       0.97
                  ts-travel-service      0.96
pod-memory-hog    ts-consign-mongo       0.96
                  ts-order-service       0.87
                  ts-preserve-service    0.53
                  ts-station-service     0.64
                  ts-train-mongo         0.51
pod-network-loss  ts-auth-mongo          1.00
                  ts-basic-service       0.90
                  ts-price-mongo         0.99
                  ts-travel-mongo        0.99
                  ts-travel2-service     0.88
dtype: float64

In [22]:
fluxrank_eval_df_sum.groupby(["chaos_type"]).mean()

chaos_type
pod-cpu-hog         0.94
pod-memory-hog      0.70
pod-network-loss    0.95
dtype: float64

In [23]:
fluxrank_eval_df_sum.mean()

0.8650375519526711

### Comparison with the existing method (SBD + Hierarchical Clustering)

In [24]:
def _sbd_hier_clustering(pk, time_series):
    metric_name_to_values = {metric: values for metric, values in time_series}
    _, clustering_info = tsdr.Tsdr("residual_integral", **{
        "step2_clustering_method_name": "hierarchy",
        "step2_hierarchy_dist_threshold": 0.02,  # should be <1.0 if 'sbd' is specified
        "step2_hierarchy_dist_type": "sbd",  # 'pearsonr' or 'sbd'
        "step2_hierarchy_linkage_method": "ward",  # 'single','complete','average','weighted', 'centroid', 'median', 'ward'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": 'medoid',  # 'medoid' or 'maxsum'

    }).reduce_multivariate_series(pd.DataFrame(metric_name_to_values), pk, n_workers=1)
    return clustering_info

sbd_hier_clustering_infos = Parallel(n_jobs=-1)(delayed(_sbd_hier_clustering)(pk, ts) for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [25]:
sbd_hier_eval_df = _create_df_from_clustering_infos(sbd_hier_clustering_infos)
sbd_hier_eval_df

index                        rep                          type_positives  type_negatives  pos_positives  pos_negatives  total_metrics
chaos_type       chaos_comp          cluster_no                                                                                                                                       
pod-memory-hog   ts-preserve-service 1              0                        s-ts-preserve_requests_count           0              0               0              0             1     
                                     2              1                  s-ts-preserve_request_errors_count           0              0               0              0             1     
                                     3              2              s-ts-preserve_request_duration_seconds           0              0               0              0             1     
                                     4              3          c-ts-preserve-service_fs_reads_bytes_total           3              0               3              0             3     
                                     5              4   m-ts-preserve-service_java_lang_Threading_Tota...           6              0               6              0             4     
                                     6              5       m-ts-preserve-service_Tomcat_Servlet_loadTime           6              0               6              0             4     
                                     7              6   c-ts-preserve-service_cpu_cfs_throttled_period...           1              0               1              0             2     
                                     8              7       c-ts-preserve-service_cpu_usage_seconds_total           1              0               1              0             2     
                                     9              8   m-ts-preserve-service_Tomcat_RequestProcessor_...          15              0              15              0             6     
                                     10             9      m-ts-preserve-service_Tomcat_WebModule_minTime           0              0               0              0             1     
                                     11            10                  c-ts-preserve-service_memory_cache           3              0               3              0             3     
                                     12            11                  c-ts-preserve-service_ulimits_soft           1              0               1              0             2     
                                     13            12        c-ts-preserve-service_memory_max_usage_bytes          10              0              10              0             5     
                                     14            13   m-ts-preserve-service_java_lang_GarbageCollect...         172             38             210              0            21     
                                     15            14   c-ts-preserve-service_network_receive_packets_...           6              0               6              0             4     
                                     16            15   m-ts-preserve-service_Tomcat_GlobalRequestProc...          28              0              28              0             8     
                                     17            16         c-ts-preserve-service_memory_failures_total           0              0               0              0             1     
                                     18            17      c-ts-preserve-service_cpu_system_seconds_total           0              0               0              0             1     
                                     19            18   m-ts-preserve-service_java_lang_GarbageCollect...          36              0              28              8             9     
                                     20            19   m-ts-preserve-service_jmx_scrape_duration_seconds           0              0               0              0             1     
                       

In [26]:
sbd_hier_eval_df_sum = sbd_hier_eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["type_positives"].sum() / (x["type_positives"].sum() + x["type_negatives"].sum()))
sbd_hier_eval_df_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.60
                  ts-order-mongo         0.67
                  ts-station-service     0.64
                  ts-train-service       0.49
                  ts-travel-service      0.74
pod-memory-hog    ts-consign-mongo       0.69
                  ts-order-service       0.84
                  ts-preserve-service    0.88
                  ts-station-service     0.81
                  ts-train-mongo         0.51
pod-network-loss  ts-auth-mongo          0.82
                  ts-basic-service       0.44
                  ts-price-mongo         0.82
                  ts-travel-mongo        0.87
                  ts-travel2-service     0.60
dtype: float64

In [27]:
sbd_hier_eval_df_sum.mean()

0.6950766485057397

In [28]:
anomaly_patterns = [
    'Sudden increase', 'Sudden decrease', 'Level shift up', 'Level shift down', 
    'Steady increase', 'Steady decrease', 'Single spike', 'Single dip',
    'Transient level shift up', 'Transient level shift down', 'Multiple spikes', 'Multiple dips', 'Fluctuations',
    'White noise', 'Other normal',
]

def _create_counting_df_from_clustering_infos(_clustering_infos: list) -> pd.DataFrame:
    eval_stat: list[tuple] = []
    for i, ((chaos_type, chaos_comp), time_series) in enumerate(time_series_by_case.items()):
        clustering_info = _clustering_infos[i]
        for i, (representative_metric, sub_metrics) in enumerate(clustering_info.items(), start=1):
            atype_positives, atype_negatives = 0, 0
            apos_positives, apos_negatives = 0, 0
            num_by_atype = {"type0": 0, "type1": 0, "type2": 0}
            num_by_apos = {"no_anomaly": 0, "anomaly_during_fault": 0, "anomaly_outside_fault": 0}
            num_by_apattern = defaultdict(int)
            for metric in [representative_metric] + sub_metrics:
                atype: str = samples[chaos_type, chaos_comp, metric]["anomaly_type"]
                apos: str = samples[chaos_type, chaos_comp, metric]["anomaly_position"]
                apattern: str = samples[chaos_type, chaos_comp, metric]["anomaly_pattern"]
                num_by_atype[atype] += 1
                num_by_apos[apos] += 1
                num_by_apattern[apattern] += 1

            eval_stat.append((chaos_type, chaos_comp, i, representative_metric, len(sub_metrics)+1, num_by_atype["type0"], num_by_atype["type1"], num_by_atype["type2"], num_by_apos["no_anomaly"], num_by_apos["anomaly_during_fault"], num_by_apos["anomaly_outside_fault"]) + tuple(num_by_apattern[p] for p in anomaly_patterns))

    return pd.DataFrame(eval_stat, columns=["chaos_type", "chaos_comp", "cluster_no", "rep", "num_metrics", "type0", "type1", "type2", "no_anomaly", "during_fault", "outside_fault"] + anomaly_patterns).reset_index().set_index(["chaos_type", "chaos_comp", "cluster_no"])

In [29]:
eval_df_count = _create_counting_df_from_clustering_infos(clustering_infos)
eval_df_count

index                        rep                          num_metrics  type0  type1  type2  no_anomaly  during_fault  outside_fault  Sudden increase  Sudden decrease  Level shift up  Level shift down  Steady increase  Steady decrease  Single spike  Single dip  Transient level shift up  Transient level shift down  Multiple spikes  Multiple dips  Fluctuations  White noise  Other normal
chaos_type       chaos_comp          cluster_no                                                                                                                                                                                                                                                                                                                                                                                                    
pod-memory-hog   ts-preserve-service 1              0              s-ts-preserve_request_duration_seconds        3        0      1      2         0            3             0               0                0                0                1                0                0               1            0                  0                          0                     1               0              0            0             0     
                                     2              1          c-ts-preserve-service_fs_reads_bytes_total        3        0      3      0         0            3             0               0                0                0                3                0                0               0            0                  0                          0                     0               0              0            0             0     
                                     3              2   m-ts-preserve-service_java_lang_GarbageCollect...        5        0      5      0         0            5             0               0                0                0                5                0                0               0            0                  0                          0                     0               0              0            0             0     
                                     4              3   m-ts-preserve-service_java_lang_Threading_Curr...       24        1      7     16         1           20             3               0                0                1                6                0                0               8            5                  0                          0                     1               0              2            0             0     
                                     5              4   m-ts-preserve-service_Tomcat_RequestProcessor_...        8        0      0      8         0            8             0               0                0                0                0                0                0               8            0                  0                          0                     0               0              0            0             0     
                                     6              5   m-ts-preserve-service_java_lang_Compilation_To...        6        0      0      6         0            6             0               0                0                0                0                0                0               6            0                  0                          0                     0               0              0            0             0     
                                     7              6            c-ts-preserve-service_memory_mapped_file        4        0      4      0         0            4             0               0                0                4                0                0                0               0            0                  0                          0                     0               0              0            0             0     
                                     8              7      c-ts-preserve-service_memory_working_set_bytes        4        0  

In [30]:
def _calculate_matching_rate(X: pd.Series) -> float:
    type0_cnt, type1_cnt, type2_cnt = X["type0"].sum(), X["type1"].sum(), X["type2"].sum()
    cnts = np.array([type0_cnt, type1_cnt, type2_cnt])
    max_idx = np.argmax(cnts)
    return cnts[max_idx] / cnts.sum()

eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_rate).groupby(["chaos_type", "chaos_comp"]).agg(["mean", "max", "min"])

mean  max   min
chaos_type       chaos_comp                          
pod-cpu-hog      ts-food-service      0.95  1.0  0.46
                 ts-order-mongo       0.93  1.0  0.50
                 ts-station-service   0.93  1.0  0.50
                 ts-train-service     0.95  1.0  0.41
                 ts-travel-service    0.94  1.0  0.50
pod-memory-hog   ts-consign-mongo     0.97  1.0  0.50
                 ts-order-service     0.96  1.0  0.56
                 ts-preserve-service  0.96  1.0  0.67
                 ts-station-service   0.92  1.0  0.45
                 ts-train-mongo       0.94  1.0  0.50
pod-network-loss ts-auth-mongo        0.97  1.0  0.50
                 ts-basic-service     0.85  1.0  0.38
                 ts-price-mongo       0.96  1.0  0.50
                 ts-travel-mongo      0.95  1.0  0.50
                 ts-travel2-service   0.91  1.0  0.50

In [31]:
fluxrank_eval_df_count = _create_counting_df_from_clustering_infos(fluxrank_clustering_infos)
fluxrank_eval_df_count

index                        rep                          num_metrics  type0  type1  type2  no_anomaly  during_fault  outside_fault  Sudden increase  Sudden decrease  Level shift up  Level shift down  Steady increase  Steady decrease  Single spike  Single dip  Transient level shift up  Transient level shift down  Multiple spikes  Multiple dips  Fluctuations  White noise  Other normal
chaos_type       chaos_comp          cluster_no                                                                                                                                                                                                                                                                                                                                                                                                    
pod-memory-hog   ts-preserve-service 1              0                        s-ts-preserve_requests_count        1        0       1      0        0             1            0               0                0                0                 1                0               0               0             0                 0                          0                     0               0              0            0             0     
                                     2              1                  s-ts-preserve_request_errors_count        1        0       0      1        0             1            0               0                0                0                 0                0               0               1             0                 0                          0                     0               0              0            0             0     
                                     3              2              s-ts-preserve_request_duration_seconds        1        0       0      1        0             1            0               0                0                0                 0                0               0               0             0                 0                          0                     1               0              0            0             0     
                                     4              3   m-ts-preserve-service_java_lang_Memory_NonHeap...       91        0      34     57        0            91            0               0                0               11                23                0               0              24            32                 0                          0                     0               0              1            0             0     
                                     5              4   m-ts-preserve-service_Tomcat_WebModule_request...       11        0      11      0        0            11            0               0                0                0                11                0               0               0             0                 0                          0                     0               0              0            0             0     
                                     6              5   m-ts-preserve-service_Tomcat_GlobalRequestProc...        3        0       0      3        0             3            0               0                0                0                 0                0               0               3             0                 0                          0                     0               0              0            0             0     
                                     7              6   m-ts-preserve-service_jmx_scrape_duration_seconds        1        0       0      1        0             1            0               0                0                0                 0                0               0               0             0                 0                          0                     1               0              0            0             0     
                                     8              7   m-ts-preserve-service_java_lang_OperatingSyste...        1        0  

In [32]:
fluxrank_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_rate).groupby(["chaos_type", "chaos_comp"]).agg(["mean", "max", "min"])

mean  max   min
chaos_type       chaos_comp                          
pod-cpu-hog      ts-food-service      1.00  1.0  0.98
                 ts-order-mongo       0.99  1.0  0.60
                 ts-station-service   0.99  1.0  0.50
                 ts-train-service     1.00  1.0  0.98
                 ts-travel-service    1.00  1.0  0.97
pod-memory-hog   ts-consign-mongo     1.00  1.0  0.67
                 ts-order-service     0.98  1.0  0.64
                 ts-preserve-service  0.98  1.0  0.63
                 ts-station-service   0.98  1.0  0.52
                 ts-train-mongo       0.98  1.0  0.54
pod-network-loss ts-auth-mongo        0.99  1.0  0.50
                 ts-basic-service     0.99  1.0  0.75
                 ts-price-mongo       1.00  1.0  1.00
                 ts-travel-mongo      0.98  1.0  0.67
                 ts-travel2-service   1.00  1.0  0.93

In [33]:
fluxrank_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_rate).mean()

0.991187436188426

In [34]:
sbd_hier_eval_df_count = _create_counting_df_from_clustering_infos(sbd_hier_clustering_infos)
sbd_hier_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_rate).groupby(["chaos_type", "chaos_comp"]).agg(["mean", "max", "min"])

mean  max   min
chaos_type       chaos_comp                          
pod-cpu-hog      ts-food-service      0.99  1.0  0.72
                 ts-order-mongo       0.98  1.0  0.45
                 ts-station-service   0.98  1.0  0.55
                 ts-train-service     0.99  1.0  0.63
                 ts-travel-service    0.98  1.0  0.55
pod-memory-hog   ts-consign-mongo     0.99  1.0  0.62
                 ts-order-service     0.98  1.0  0.41
                 ts-preserve-service  0.99  1.0  0.85
                 ts-station-service   0.98  1.0  0.55
                 ts-train-mongo       0.98  1.0  0.48
pod-network-loss ts-auth-mongo        0.97  1.0  0.50
                 ts-basic-service     0.98  1.0  0.59
                 ts-price-mongo       1.00  1.0  0.90
                 ts-travel-mongo      0.99  1.0  0.62
                 ts-travel2-service   0.96  1.0  0.50

In [35]:
sbd_hier_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_rate).mean()

0.9834397254680319

## Mismatch count

In [36]:
def _calculate_matching_count(X: pd.Series) -> bool:
    type0_cnt, type1_cnt, type2_cnt = X["type0"].sum(), X["type1"].sum(), X["type2"].sum()
    cnts = np.array([type0_cnt, type1_cnt, type2_cnt])
    max_idx = np.argmax(cnts)
    return cnts[max_idx] == cnts.sum()

eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.83
                  ts-order-mongo         0.78
                  ts-station-service     0.82
                  ts-train-service       0.85
                  ts-travel-service      0.81
pod-memory-hog    ts-consign-mongo       0.91
                  ts-order-service       0.89
                  ts-preserve-service    0.86
                  ts-station-service     0.79
                  ts-train-mongo         0.80
pod-network-loss  ts-auth-mongo          0.91
                  ts-basic-service       0.62
                  ts-price-mongo         0.84
                  ts-travel-mongo        0.83
                  ts-travel2-service     0.75
dtype: float64

In [37]:
fluxrank_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.99
                  ts-order-mongo         0.95
                  ts-station-service     0.96
                  ts-train-service       0.99
                  ts-travel-service      0.98
pod-memory-hog    ts-consign-mongo       0.97
                  ts-order-service       0.91
                  ts-preserve-service    0.94
                  ts-station-service     0.95
                  ts-train-mongo         0.96
pod-network-loss  ts-auth-mongo          0.96
                  ts-basic-service       0.94
                  ts-price-mongo         0.98
                  ts-travel-mongo        0.88
                  ts-travel2-service     0.99
dtype: float64

In [38]:
sbd_hier_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.95
                  ts-order-mongo         0.94
                  ts-station-service     0.93
                  ts-train-service       0.97
                  ts-travel-service      0.94
pod-memory-hog    ts-consign-mongo       0.96
                  ts-order-service       0.92
                  ts-preserve-service    0.94
                  ts-station-service     0.94
                  ts-train-mongo         0.93
pod-network-loss  ts-auth-mongo          0.93
                  ts-basic-service       0.95
                  ts-price-mongo         0.96
                  ts-travel-mongo        0.94
                  ts-travel2-service     0.91
dtype: float64

### by anomaly position

In [39]:
def _calculate_matching_count_anomaly_pos(X: pd.Series) -> bool:
    cnts = np.array([X["no_anomaly"].sum(), X["during_fault"].sum(), X["outside_fault"].sum()])
    max_idx = np.argmax(cnts)
    return cnts[max_idx] == cnts.sum()

## no_anomaly	during_fault	outside_fault
eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count_anomaly_pos).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.73
                  ts-order-mongo         0.81
                  ts-station-service     0.76
                  ts-train-service       0.80
                  ts-travel-service      0.78
pod-memory-hog    ts-consign-mongo       0.82
                  ts-order-service       0.87
                  ts-preserve-service    0.95
                  ts-station-service     0.81
                  ts-train-mongo         0.70
pod-network-loss  ts-auth-mongo          0.91
                  ts-basic-service       0.62
                  ts-price-mongo         0.77
                  ts-travel-mongo        0.86
                  ts-travel2-service     0.77
dtype: float64

In [40]:
fluxrank_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count_anomaly_pos).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.99
                  ts-order-mongo         0.96
                  ts-station-service     0.96
                  ts-train-service       0.96
                  ts-travel-service      0.97
pod-memory-hog    ts-consign-mongo       0.96
                  ts-order-service       0.93
                  ts-preserve-service    1.00
                  ts-station-service     0.97
                  ts-train-mongo         0.96
pod-network-loss  ts-auth-mongo          0.92
                  ts-basic-service       0.91
                  ts-price-mongo         0.96
                  ts-travel-mongo        0.92
                  ts-travel2-service     0.95
dtype: float64

In [41]:
sbd_hier_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count_anomaly_pos).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.92
                  ts-order-mongo         0.93
                  ts-station-service     0.91
                  ts-train-service       0.98
                  ts-travel-service      0.98
pod-memory-hog    ts-consign-mongo       0.96
                  ts-order-service       0.95
                  ts-preserve-service    0.97
                  ts-station-service     0.96
                  ts-train-mongo         0.93
pod-network-loss  ts-auth-mongo          0.93
                  ts-basic-service       0.95
                  ts-price-mongo         0.96
                  ts-travel-mongo        0.94
                  ts-travel2-service     0.91
dtype: float64

In [42]:
fluxrank_sbd_eval_df_count = _create_counting_df_from_clustering_infos(fluxrank_sbd_clustering_infos)
fluxrank_sbd_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.91
                  ts-order-mongo         0.86
                  ts-station-service     0.88
                  ts-train-service       0.94
                  ts-travel-service      0.92
pod-memory-hog    ts-consign-mongo       0.93
                  ts-order-service       0.92
                  ts-preserve-service    0.86
                  ts-station-service     0.92
                  ts-train-mongo         0.93
pod-network-loss  ts-auth-mongo          0.89
                  ts-basic-service       0.91
                  ts-price-mongo         0.87
                  ts-travel-mongo        0.87
                  ts-travel2-service     0.84
dtype: float64

In [44]:
fluxrank_sbd_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X)).agg(["mean", "max", "min"])

mean    0.90
max     0.94
min     0.84
dtype: float64

In [45]:
fluxrank_sbd_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count_anomaly_pos).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.86
                  ts-order-mongo         0.86
                  ts-station-service     0.84
                  ts-train-service       0.94
                  ts-travel-service      0.88
pod-memory-hog    ts-consign-mongo       0.82
                  ts-order-service       0.85
                  ts-preserve-service    0.86
                  ts-station-service     0.88
                  ts-train-mongo         0.90
pod-network-loss  ts-auth-mongo          0.89
                  ts-basic-service       0.91
                  ts-price-mongo         0.87
                  ts-travel-mongo        0.87
                  ts-travel2-service     0.84
dtype: float64

In [46]:
pearsonr_hdbscan_eval_df_count = _create_counting_df_from_clustering_infos(pearsonr_clustering_infos)
pearsonr_hdbscan_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.92
                  ts-order-mongo         0.92
                  ts-station-service     0.89
                  ts-train-service       0.86
                  ts-travel-service      0.92
pod-memory-hog    ts-consign-mongo       0.90
                  ts-order-service       0.94
                  ts-preserve-service    0.86
                  ts-station-service     0.85
                  ts-train-mongo         0.88
pod-network-loss  ts-auth-mongo          0.93
                  ts-basic-service       0.55
                  ts-price-mongo         0.95
                  ts-travel-mongo        0.88
                  ts-travel2-service     0.80
dtype: float64

In [47]:
pearsonr_hdbscan_eval_df_count.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_matching_count_anomaly_pos).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.87
                  ts-order-mongo         0.91
                  ts-station-service     0.83
                  ts-train-service       0.83
                  ts-travel-service      0.87
pod-memory-hog    ts-consign-mongo       0.90
                  ts-order-service       0.90
                  ts-preserve-service    0.95
                  ts-station-service     0.92
                  ts-train-mongo         0.88
pod-network-loss  ts-auth-mongo          0.93
                  ts-basic-service       0.45
                  ts-price-mongo         0.93
                  ts-travel-mongo        0.89
                  ts-travel2-service     0.78
dtype: float64

## Faulty metrics based evaluation

In [48]:
# Loading reduced metrics data
import pathlib
import pickle
import random

def load_tsdr():
    results = []
    parent_path = pathlib.Path(f"../data/tsdr_rq54b")
    for path in parent_path.iterdir():
        with (path / "record.pkl").open("rb") as f:
            record = pickle.load(f)
        with (path / "reduced_df.pkl").open("rb") as f:
            reduced_df = pickle.load(f)
        with (path / "no_clustering_reduced_df.pkl").open("rb") as f:
            no_clustering_reduced_df = pickle.load(f)
        results.append((record, reduced_df, no_clustering_reduced_df))
    return results

datasets = load_tsdr()

In [50]:
# Find faulty metrics
from eval.groundtruth import check_cause_metrics

faulty_metrics_by_fault_case = {}
for record, _, _ in datasets:
    faulty_metrics_by_fault_case[record.chaos_type(), record.chaos_comp()] = check_cause_metrics(pk, record.metrics_names(), record.chaos_type(), record.chaos_comp())[1]

In [79]:
def _create_faulty_metrics_df_from_clustering_infos(_clustering_infos: list) -> pd.DataFrame:
    eval_stat: list[tuple] = []
    for i, ((chaos_type, chaos_comp), time_series) in enumerate(time_series_by_case.items()):
        clustering_info = _clustering_infos[i]
        faulty_metrics = faulty_metrics_by_fault_case[(chaos_type, chaos_comp)].tolist()
        for i, (representative_metric, sub_metrics) in enumerate(clustering_info.items(), start=1):
            num_by_atype = {"type0": 0, "type1": 0, "type2": 0}
            faulty_metrics_by_atype = {"type0": 0, "type1": 0, "type2": 0} 
            # num_by_apos = {"no_anomaly": 0, "anomaly_during_fault": 0, "anomaly_outside_fault": 0}
            # num_by_apattern = defaultdict(int)
            representative_metric_type = samples[chaos_type, chaos_comp, representative_metric]["anomaly_type"]
            for metric in [representative_metric] + sub_metrics:
                atype: str = samples[chaos_type, chaos_comp, metric]["anomaly_type"]
                num_by_atype[atype] += 1
                if metric in faulty_metrics:
                    faulty_metrics_by_atype[atype] += 1
                # apos: str = samples[chaos_type, chaos_comp, metric]["anomaly_position"]
                # apattern: str = samples[chaos_type, chaos_comp, metric]["anomaly_pattern"]

            eval_stat.append((chaos_type, chaos_comp, i, representative_metric, representative_metric_type, len(sub_metrics)+1, num_by_atype["type0"], num_by_atype["type1"], num_by_atype["type2"], faulty_metrics_by_atype["type0"], faulty_metrics_by_atype["type1"], faulty_metrics_by_atype["type2"]))

    return pd.DataFrame(eval_stat, columns=["chaos_type", "chaos_comp", "cluster_no", "rep", "rep_type", "num_metrics", "type0", "type1", "type2", "faulty_type0", "faulty_type1", "faulty_type2"]).reset_index().set_index(["chaos_type", "chaos_comp", "cluster_no"])

In [80]:
eval_faulty_metrics_df = _create_faulty_metrics_df_from_clustering_infos(clustering_infos)
eval_faulty_metrics_df

index                        rep                         rep_type  num_metrics  type0  type1  type2  faulty_type0  faulty_type1  faulty_type2
chaos_type       chaos_comp          cluster_no                                                                                                                                               
pod-memory-hog   ts-preserve-service 1              0              s-ts-preserve_request_duration_seconds   type2         3        0      1      2          0             0             0     
                                     2              1          c-ts-preserve-service_fs_reads_bytes_total   type1         3        0      3      0          0             0             0     
                                     3              2   m-ts-preserve-service_java_lang_GarbageCollect...   type1         5        0      5      0          0             1             0     
                                     4              3   m-ts-preserve-service_java_lang_Threading_Curr...   type2        24        1      7     16          0             4             6     
                                     5              4   m-ts-preserve-service_Tomcat_RequestProcessor_...   type2         8        0      0      8          0             0             2     
                                     6              5   m-ts-preserve-service_java_lang_Compilation_To...   type2         6        0      0      6          0             0             1     
                                     7              6            c-ts-preserve-service_memory_mapped_file   type1         4        0      4      0          0             1             0     
                                     8              7      c-ts-preserve-service_memory_working_set_bytes   type1         4        0      4      0          0             3             0     
                                     9              8                     c-ts-preserve-service_last_seen   type1         3        0      3      0          0             0             0     
                                     10             9   c-ts-preserve-service_network_receive_packets_...   type1         4        0      4      0          0             0             0     
                                     11            10   m-ts-preserve-service_Tomcat_Servlet_requestCount   type1         4        0      4      0          0             3             0     
                                     12            11   m-ts-preserve-service_Tomcat_GlobalRequestProc...   type1         3        0      3      0          0             3             0     
                                     13            12   m-ts-preserve-service_java_lang_GarbageCollect...   type2         5        0      0      5          0             0             3     
                                     14            13   m-ts-preserve-service_Tomcat_GlobalRequestProc...   type2         3        0      0      3          0             0             0     
                                     15            14    m-ts-preserve-service_java_lang_MemoryPool_Valid   type2        11        0      0     11          0             0             5     
                                     16            15   m-ts-preserve-service_java_lang_ClassLoading_L...   type2         4        0      0      4          0             0             0     
                                     17            16   m-ts-preserve-service_Tomcat_WebResourceRoot_l...   type2         3        0      0      3          0             0             1     
                                     18            17   m-ts-preserve-service_java_lang_GarbageCollect...   type1         8        0      6      2          0             4             2     
                                     19            18   m-ts-preserve-service_java_lang_Threading_Thre...   type2        10        0      0     10          0             0             0     
                                     2

In [87]:
def _calculate_faulty_type(X: pd.Series) -> pd.Series:
    types = ["type0", "type1", "type2"]

    type_cnts = np.array([X[t].sum() for t in types])
    max_idx = np.argmax(type_cnts)

    faulty_type_cnts = np.array([X[f"faulty_{t}"].sum() for t in types])
    faulty_max_idx = np.argmax(faulty_type_cnts)
    if faulty_type_cnts[max_idx] == 0:
        # in case of faulty metrics are not found
        return pd.Series({"faulty_type": True, "faulty_rep_type": True})

    res = {"faulty_type": max_idx == faulty_max_idx, "faulty_rep_type": X["rep_type"][0] == types[faulty_max_idx]}
    return pd.Series(res)


eval_faulty_metrics_df.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_faulty_type).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

AttributeError: 'bool' object has no attribute 'index'

In [70]:
fluxrank_eval_faulty_metrics_df = _create_faulty_metrics_df_from_clustering_infos(fluxrank_clustering_infos)
fluxrank_eval_faulty_metrics_df.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_faulty_type).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        1.00
                  ts-order-mongo         1.00
                  ts-station-service     1.00
                  ts-train-service       1.00
                  ts-travel-service      1.00
pod-memory-hog    ts-consign-mongo       1.00
                  ts-order-service       1.00
                  ts-preserve-service    0.94
                  ts-station-service     1.00
                  ts-train-mongo         1.00
pod-network-loss  ts-auth-mongo          1.00
                  ts-basic-service       1.00
                  ts-price-mongo         1.00
                  ts-travel-mongo        1.00
                  ts-travel2-service     1.00
dtype: float64

In [72]:
fluxrank_sbd_eval_faulty_metrics_df = _create_faulty_metrics_df_from_clustering_infos(fluxrank_sbd_clustering_infos)
fluxrank_sbd_eval_faulty_metrics_df.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_faulty_type).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        1.00
                  ts-order-mongo         1.00
                  ts-station-service     1.00
                  ts-train-service       1.00
                  ts-travel-service      1.00
pod-memory-hog    ts-consign-mongo       1.00
                  ts-order-service       1.00
                  ts-preserve-service    0.86
                  ts-station-service     0.96
                  ts-train-mongo         1.00
pod-network-loss  ts-auth-mongo          1.00
                  ts-basic-service       0.91
                  ts-price-mongo         1.00
                  ts-travel-mongo        1.00
                  ts-travel2-service     1.00
dtype: float64

In [73]:
pearsonr_eval_faulty_metrics_df = _create_faulty_metrics_df_from_clustering_infos(pearsonr_clustering_infos)
pearsonr_eval_faulty_metrics_df.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_faulty_type).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        1.00
                  ts-order-mongo         1.00
                  ts-station-service     1.00
                  ts-train-service       1.00
                  ts-travel-service      1.00
pod-memory-hog    ts-consign-mongo       1.00
                  ts-order-service       1.00
                  ts-preserve-service    1.00
                  ts-station-service     0.98
                  ts-train-mongo         0.97
pod-network-loss  ts-auth-mongo          1.00
                  ts-basic-service       0.91
                  ts-price-mongo         1.00
                  ts-travel-mongo        1.00
                  ts-travel2-service     1.00
dtype: float64

In [74]:
sbd_hier_eval_faulty_metrics_df = _create_faulty_metrics_df_from_clustering_infos(sbd_hier_clustering_infos)
sbd_hier_eval_faulty_metrics_df.groupby(["chaos_type", "chaos_comp", "cluster_no"]).apply(_calculate_faulty_type).groupby(["chaos_type", "chaos_comp"]).apply(lambda X: X.sum() / len(X))

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        1.00
                  ts-order-mongo         1.00
                  ts-station-service     1.00
                  ts-train-service       1.00
                  ts-travel-service      1.00
pod-memory-hog    ts-consign-mongo       1.00
                  ts-order-service       1.00
                  ts-preserve-service    1.00
                  ts-station-service     1.00
                  ts-train-mongo         1.00
pod-network-loss  ts-auth-mongo          1.00
                  ts-basic-service       0.95
                  ts-price-mongo         1.00
                  ts-travel-mongo        1.00
                  ts-travel2-service     1.00
dtype: float64